In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Reshape, Input, LSTM, Conv1D, MaxPooling1D, Flatten

In [2]:
Train = pd.read_csv('dataTrain.csv')
Test = pd.read_csv('dataTest.csv')

X_train = Train.drop('STATUS', axis=1)
y_train = Train['STATUS']

X_test = Test.drop('STATUS', axis=1)
y_test = Test['STATUS']

In [3]:
def train_dnn(X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    
    accuracies = []
    recalls = []
    precisions = []
    f1_scores = []
    kappas = []

    model = Sequential([
        Input(shape=(X.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy')

    for train_index, test_index in kf.split(X):
        X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
        
        model.set_weights([np.random.permutation(w.flatten()).reshape(w.shape) for w in model.get_weights()])

        model.fit(X_train_fold, y_train_fold, epochs=20, batch_size=32, verbose=0)
        
        y_pred = (model.predict(X_test_fold, verbose=0) > 0.5).astype("int32")

        accuracies.append(accuracy_score(y_test_fold, y_pred))
        recalls.append(recall_score(y_test_fold, y_pred))
        precisions.append(precision_score(y_test_fold, y_pred))
        f1_scores.append(f1_score(y_test_fold, y_pred))
        kappas.append(cohen_kappa_score(y_test_fold, y_pred))


    print("DNN Results:")
    print(f"Average Accuracy: {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")
    print(f"Average Recall: {np.mean(recalls):.3f} ± {np.std(recalls):.3f}")
    print(f"Average Precision: {np.mean(precisions):.3f} ± {np.std(recalls):.3f}")
    print(f"Average F1-Score: {np.mean(f1_scores):.3f} ± {np.std(f1_scores):.3f}")
    print(f"Average Kappa: {np.mean(kappas):.3f} ± {np.std(kappas):.3f}\n")

train_dnn(X_train, y_train)

DNN Results:
Average Accuracy: 0.957 ± 0.004
Average Recall: 0.947 ± 0.013
Average Precision: 0.965 ± 0.013
Average F1-Score: 0.956 ± 0.004
Average Kappa: 0.914 ± 0.007



In [4]:
def train_mlp(hidden_layer_sizes, X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    
    accuracies = []
    recalls = []
    precisions = []
    f1_scores = []
    kappas = []
    
    for train_index, test_index in kf.split(X):
        X_train2, X_test2 = X.iloc[train_index], X.iloc[test_index]
        y_train2, y_test2 = y.iloc[train_index], y.iloc[test_index]

        model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, random_state=1, max_iter=5000)
        model.fit(X_train2, y_train2)

        y_pred = model.predict(X_test2)

        accuracies.append(accuracy_score(y_test2, y_pred))
        recalls.append(recall_score(y_test2, y_pred, average='weighted'))
        precisions.append(precision_score(y_test2, y_pred, average='weighted'))
        f1_scores.append(f1_score(y_test2, y_pred, average='weighted'))
        kappas.append(cohen_kappa_score(y_test2, y_pred))


    print(f'MLP Architecture: {hidden_layer_sizes}')
    print(f"Average Accuracy: {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")
    print(f"Average Recall: {np.mean(recalls):.3f} ± {np.std(recalls):.3f}")
    print(f"Average Precision: {np.mean(precisions):.3f} ± {np.std(precisions):.3f}")
    print(f"Average F1-Score: {np.mean(f1_scores):.3f} ± {np.std(f1_scores):.3f}")
    print(f"Average Kappa: {np.mean(kappas):.3f} ± {np.std(kappas):.3f}\n")

Architectures = [[5], [10], [5, 5], [5, 10], [10, 10]]
for arch in Architectures:
    train_mlp(arch, X_train, y_train)



MLP Architecture: [5]
Average Accuracy: 0.916 ± 0.011
Average Recall: 0.916 ± 0.011
Average Precision: 0.916 ± 0.011
Average F1-Score: 0.916 ± 0.011
Average Kappa: 0.831 ± 0.023

MLP Architecture: [10]
Average Accuracy: 0.935 ± 0.004
Average Recall: 0.935 ± 0.004
Average Precision: 0.935 ± 0.004
Average F1-Score: 0.935 ± 0.004
Average Kappa: 0.870 ± 0.007

MLP Architecture: [5, 5]
Average Accuracy: 0.926 ± 0.005
Average Recall: 0.926 ± 0.005
Average Precision: 0.926 ± 0.005
Average F1-Score: 0.926 ± 0.005
Average Kappa: 0.852 ± 0.010

MLP Architecture: [5, 10]
Average Accuracy: 0.924 ± 0.005
Average Recall: 0.924 ± 0.005
Average Precision: 0.924 ± 0.005
Average F1-Score: 0.924 ± 0.005
Average Kappa: 0.849 ± 0.010

MLP Architecture: [10, 10]
Average Accuracy: 0.950 ± 0.005
Average Recall: 0.950 ± 0.005
Average Precision: 0.950 ± 0.005
Average F1-Score: 0.950 ± 0.005
Average Kappa: 0.901 ± 0.009

